In [63]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [87]:
train_df = pd.read_csv('credit_train.csv')
train_df = train_df.rename(columns={
    'Loan ID': 'loan_id',
    'Customer ID': 'customer_id',
    'Loan Status': 'loan_status',
    'Current Loan Amount': 'current_loan_amount',
    'Term': 'term',
    'Credit Score': 'credit_score',
    'Annual Income': 'annual_income',
    'Years in current job': 'years_in_current_job',
    'Home Ownership': 'ownership',
    'Purpose': 'purpose',
    'Monthly Debt': 'monthly_debt',
    'Years of Credit History': 'years_of_credit_history',
    'Months since last delinquent': 'months',
    'Number of Open Accounts': 'open_accounts',
    'Number of Credit Problems': 'credit_problems',
    'Current Credit Balance': 'credit_balance',
    'Maximum Open Credit': 'max_credit',
    'Bankrupticies': 'bankruptcy',
    'Tax Liens': 'tax_liens'
})

train_df = train_df.drop_duplicates()

train_df

,loan_id,customer_id,loan_status,current_loan_amount,term,credit_score,annual_income,years_in_current_job,ownership,purpose,monthly_debt,years_of_credit_history,months,open_accounts,credit_problems,credit_balance,max_credit,Bankruptcies,tax_liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6,1,228190,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35,0,229976,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9,0,256329,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15,0,253460,427174.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,8506a4e9-af7d-47d2-a1bf-7ea2c41858f0,be67200e-1ef1-4b63-86a6-2bf27d3c704d,Fully Paid,210584,Short Term,719.0,783389.0,1 year,Home Mortgage,Other,3727.61,17.4,18.0,6,0,456,259160.0,0.0,0.0
99996,06eba04f-58fc-424a-b666-ed72aa008900,77f2252a-b7d1-4b07-a746-1202a8304290,Fully Paid,99999999,Short Term,732.0,1289416.0,1 year,Rent,Debt Consolidation,13109.05,9.4,21.0,22,0,153045,509234.0,0.0,0.0
99997,e1cb4050-eff5-4bdb-a1b0-aabd3f7eaac7,2ced5f10-bd60-4a11-9134-cadce4e7b0a3,Fully Paid,103136,Short Term,742.0,1150545.0,6 years,Rent,Debt Consolidation,7315.57,18.8,18.0,12,1,109554,537548.0,1.0,0.0
99998,81ab928b-d1a5-4523-9a3c-271ebb01b4fb,3e45ffda-99fd-4cfc-b8b8-446f4a505f36,Fully Paid,530332,Short Term,746.0,1717524.0,9 years,Rent,Debt Consolidation,9890.07,15.0,NaN,8,0,404225,738254.0,0.0,0.0


In [88]:
# for column in train_df.filter(items=['loan_status', 'term', 'years_in_current_job', 'ownership', 'purpose']).columns:
#     print(f'{column}: {train_df[column].unique()}\n')
    
def ownership_resolver(ownership):
    if ownership in ['Home Mortgage', 'HaveMortgage']:
        return 'mortgage'
    else:
        return ownership
    
def purpose_resolver(purpose):
    if purpose in ['other', 'Other']:
        return 'other'
    elif purpose in ['Take a Trip', 'vacation']:
        return 'vacation'
    elif purpose in ['Buy House', 'Buy a Car', 'major_purchase']:
        return 'major_purchase'
    elif purpose in ['Home Improvements', 'moving']:
        return 'moving'
    else:
        return purpose

print(train_df.ownership.unique())
    
train_df.purpose = train_df.purpose.apply(purpose_resolver)
train_df.ownership = train_df.ownership.apply(ownership_resolver)

print(train_df.ownership.unique())

['Home Mortgage' 'Own Home' 'Rent' 'HaveMortgage']
['mortgage' 'Own Home' 'Rent']


In [89]:
train_df = train_df.drop(labels=['loan_id', 'customer_id'], axis=1)

labels = ['loan_status', 'term', 'years_in_current_job', 'ownership', 'purpose']

for label in labels:
    replaced = list(train_df[label].unique())
    train_df[label] = train_df[label].apply(lambda x: replaced.index(x))

# train_df.fillna(value=-1, inplace=True)
train_df.dropna(inplace=True)

train_df

,loan_status,current_loan_amount,term,credit_score,annual_income,years_in_current_job,ownership,purpose,monthly_debt,years_of_credit_history,months,open_accounts,credit_problems,credit_balance,max_credit,Bankruptcies,tax_liens
2,0,99999999,0,741.0,2231892.0,0,1,1,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
6,0,217646,0,730.0,1184194.0,4,0,1,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
8,0,548746,0,678.0,2559110.0,5,2,1,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
10,0,99999999,0,728.0,714628.0,2,2,1,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
12,0,99999999,0,740.0,776188.0,4,1,1,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,0,179454,0,708.0,1356068.0,6,0,1,22318.54,22.7,21.0,25,0,387410,1705616.0,0.0,0.0
99990,0,99999999,0,742.0,1190046.0,4,2,3,11969.81,20.1,16.0,9,0,37392,134442.0,0.0,0.0
99994,0,210584,0,719.0,783389.0,9,0,3,3727.61,17.4,18.0,6,0,456,259160.0,0.0,0.0
99996,0,99999999,0,732.0,1289416.0,9,2,1,13109.05,9.4,21.0,22,0,153045,509234.0,0.0,0.0


In [90]:
X = train_df.drop(labels=['loan_status'], axis=1)
y = train_df.loan_status

In [91]:
lr = LogisticRegression(penalty='l1', solver='liblinear', max_iter=100000)
lr.fit(X, y)

lr.score(X, y)

0.8215742188697759

In [92]:
test_df = pd.read_csv('credit_test.csv')
test_df = test_df.rename(columns={
    'Loan ID': 'loan_id',
    'Customer ID': 'customer_id',
    'Current Loan Amount': 'current_loan_amount',
    'Term': 'term',
    'Credit Score': 'credit_score',
    'Annual Income': 'annual_income',
    'Years in current job': 'years_in_current_job',
    'Home Ownership': 'ownership',
    'Purpose': 'purpose',
    'Monthly Debt': 'monthly_debt',
    'Years of Credit History': 'years_of_credit_history',
    'Months since last delinquent': 'months',
    'Number of Open Accounts': 'open_accounts',
    'Number of Credit Problems': 'credit_problems',
    'Current Credit Balance': 'credit_balance',
    'Maximum Open Credit': 'max_credit',
    'Bankrupticies': 'bankruptcy',
    'Tax Liens': 'tax_liens'
})

# for column in test_df.filter(items=['loan_status', 'term', 'years_in_current_job', 'ownership', 'purpose']).columns:
#     print(f'{column}: {test_df[column].unique()}\n')

# test_df = test_df.drop(labels=['loan_id', 'customer_id'], axis=1)

test_df.purpose = test_df.purpose.apply(purpose_resolver)
test_df.ownership = test_df.ownership.apply(ownership_resolver)

labels = ['term', 'years_in_current_job', 'ownership', 'purpose']
for label in labels:
    replaced = list(test_df[label].unique())
    test_df[label] = test_df[label].apply(lambda x: replaced.index(x))

test_df.fillna(value=-1, inplace=True)

predicted = lr.predict(test_df.drop(labels=['loan_id', 'customer_id'], axis=1))

In [93]:
import csv

with open('result4.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Loan ID', 'Loan Status'])
    
    for loan_id, res in zip(test_df['loan_id'], predicted):
        s_res = 'Fully Paid' if res == 0 else 'Charged Off'
            
        writer.writerow([loan_id, s_res])

In [96]:
import seaborn as sns
import matplotlib.pyplot as plt

In [101]:
print(np.unique(predicted, return_counts=True))
train_df.loan_status.value_counts()

(array([0, 1]), array([9544,  456]))


0    24722
1     7891
Name: loan_status, dtype: int64

In [102]:
df = pd.read_csv('credit_train.csv')
df['Loan Status'].value_counts()

Fully Paid     77361
Charged Off    22639
Name: Loan Status, dtype: int64